In [1]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig

In [2]:
dataset = load_dataset("iamjoon/finance_news_summarizer", split="train")

In [3]:
print("전체 데이터 크기:", len(dataset))

전체 데이터 크기: 991


In [4]:
test_ratio = 0.5

train_data = []
test_data = []

In [ ]:
# 데이터셋 인덱스를 0부터 데이터 개수-1까지 리스트로 만든다
data_indices = list(range(len(dataset)))
# 테스트 사이즈를 구한다
test_size = int(len(dataset) * test_ratio)

# 데이터셋 인덱스 지정
test_data = data_indices[:test_size]
train_data = data_indices[test_size:]

In [ ]:
# openai format으로 데이터 변환
def format_data(sample):
    return {
        "messages":[
            {
                "role": "system",
                "content": sample["system_prompt"]
            },
            {
                "role": "user",
                "content": sample["user_prompt"]
            },
            # 정답 예시 제공 json타입이기 때문에 str타입으로 변환 후 제공
            {
                "role": "assistant",
                "content": str(sample["assistant"])
            }
        ]
    }

In [7]:
# 데이터셋 OpenAI format으로 변환
train_dataset = [format_data(dataset[i]) for i in train_data]
test_dataset = [format_data(dataset[i]) for i in test_data]

In [8]:
# 최종 데이터셋 크기 출력
print("훈련 데이터 크기:", len(train_dataset))
print("테스트 데이터 크기:", len(test_dataset))

훈련 데이터 크기: 496
테스트 데이터 크기: 495


In [9]:
train_dataset[345]["messages"]

[{'role': 'system',
  'content': '당신은 주어진 뉴스로부터 종목에 영향을 주는 뉴스인지 판별하는 금융 뉴스 판별기입니다.\n두 가지 답변 케이스가 존재하며 무조건 파이썬의 dictionary 형식으로 작성하십시오.\n큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다. 따라서 주의하십시오.\n아래 dictionary에서 각 value는 지시사항에 해당합니다. 지사사항을 따라 적지마십시오. 해당 지시사항에 따라 적절한 value를 채워넣으십시오.\n해당사항이 없다면 빈 문자열 또는 빈 리스트로 적어야 합니다. 임의로 \'없음\' 등을 적어서는 안 됩니다.\n\n만약 해당 뉴스가 특정 종목(회사)이 언급되지 않거나, 특정 종목(회사)와 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.\n\n답변:\n{"is_stock_related": False,\n"summary": "여기에는 해당 뉴스를 요약해서 요약문을 작성하십시오"}\n\n만약 해당 뉴스가 특정 종목(회사)들과 연관되었거나, 특정 종목(회사)과 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.\n\n답변:\n{"is_stock_related": True,\n"positive_impact_stocks": ["파이썬 문자열 리스트의 형태로 이 뉴스가 긍정적인 영향을 줄것으로 추정되는 종목들의 이름을 작성하십시오. 약자로 적거나 별명으로 적지마십시오. 종목명으로 추정되는 한글명을 적으십시오. 뉴스로부터 추정할 수 있는 정확한 풀네임으로 적으십시오. 만약, 존재하지 않는다면 빈 리스트로 작성하십시오."],\n"reason_for_positive_impact": "위의 종목들이 해당 뉴스로부터 긍정적인 영향을 받을 것으로 추정한 이유를 여기에다가 작성하십시오",\n"positive_keywords": ["긍정적인 영향을 줄 것으로 추정되는 종목들이 존재했다면 여기에 긍정적인 영향을 주는데 근거가 